### Trabajo libre. (Feature)
#### Para el presente trabajo se tomo siete estrofas del Himno nacional con el fin de encontrar similitudes. entre estas. 

In [1]:
import pandas as pd

In [2]:
himno2=pd.read_csv(r'C:/Users/57314/Mis documentos/Himno.csv')

In [3]:
himno2

,estrofa 1,estrofa 2,estrofa 3,estrofa 4,estrofa 5,estrofa 6,estrofa 7
0,"¡Oh, gloria inmarcesible! ¡Oh, júbilo inmortal...",¡Independencia! grita El mundo americano; Se b...,Del Orinoco el cauce Se colma de despojos; De ...,A orillas del Caribe Hambriento un pueblo luch...,De Boyacá en los campos El genio de la gloria ...,Bolívar cruza el Ande Que riega dos océanos; E...,"La trompa victoriosa Que en Ayacucho truena, E..."


In [4]:
himno=himno2.T
#himno
#himno.rename(columns={'0': 'letra'}, inplace=True)
himno.columns = ['Letra']


In [5]:
himno['Letra']

estrofa 1    ¡Oh, gloria inmarcesible! ¡Oh, júbilo inmortal...
estrofa 2    ¡Independencia! grita El mundo americano; Se b...
estrofa 3    Del Orinoco el cauce Se colma de despojos; De ...
estrofa 4    A orillas del Caribe Hambriento un pueblo luch...
estrofa 5    De Boyacá en los campos El genio de la gloria ...
estrofa 6    Bolívar cruza el Ande Que riega dos océanos; E...
estrofa 7    La trompa victoriosa Que en Ayacucho truena, E...
Name: Letra, dtype: object

### Pre procesamiento

In [6]:

import re
import pandas as pd

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    return texto.split()

himno['Letra_pre'] = himno['Letra'].apply(lambda texto: pre_procesado(texto))
himno


,Letra,Letra_pre
estrofa 1,"¡Oh, gloria inmarcesible! ¡Oh, júbilo inmortal...","[oh, gloria, inmarcesible, oh, júbilo, inmorta..."
estrofa 2,¡Independencia! grita El mundo americano; Se b...,"[independencia, grita, mundo, americano, baña,..."
estrofa 3,Del Orinoco el cauce Se colma de despojos; De ...,"[orinoco, cauce, colma, despojos, sangre, llan..."
estrofa 4,A orillas del Caribe Hambriento un pueblo luch...,"[orillas, caribe, hambriento, pueblo, lucha, h..."
estrofa 5,De Boyacá en los campos El genio de la gloria ...,"[boyacá, campos, genio, gloria, cada, espiga, ..."
estrofa 6,Bolívar cruza el Ande Que riega dos océanos; E...,"[bolívar, cruza, ande, riega, dos, océanos, es..."
estrofa 7,"La trompa victoriosa Que en Ayacucho truena, E...","[trompa, victoriosa, ayacucho, truena, cada, t..."


In [7]:
! pip install gensim

In [8]:

import gensim.models.word2vec as w2v

In [67]:
mi_modelo = w2v.Word2Vec(himno['Letra'].values,
                            sg=2, # 1 para usar skip-gram, 0 para usar CBOW
                            seed=1, # 1 para tener resultados reproducibles y debugear
                            size=256, # el tamaño de los vectores de palabras, o neuronas
                            min_count=1, #mínimo de veces que ha de aparecer cada palabra, para ser considerada,
                            window=12)

In [74]:
mi_modelo.wv.most_similar("gloria")
### No se pudo

KeyError: "word 'gloria' not in vocabulary"

In [48]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import iplot

###  Miraremos la distancia entre estrofas.


In [57]:
himno['Estrofa'] = list(himno.index.values.tolist())

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(himno['Letra'].values)

tfidf_matrix = pd.DataFrame(data=tfidf.toarray(), columns=tfidf_vect.get_feature_names())

tfidf_matrix = tfidf_matrix.T.round(3)
tfidf_matrix.columns = himno.Estrofa

tfidf_matrix

Estrofa,estrofa 1,estrofa 2,estrofa 3,estrofa 4,estrofa 5,estrofa 6,estrofa 7
abnegación,0.000,0.0,0.000,0.217,0.000,0.0,0.000
admiración,0.000,0.0,0.197,0.000,0.000,0.0,0.000
aliento,0.000,0.0,0.000,0.000,0.205,0.0,0.000
allí,0.000,0.0,0.197,0.000,0.000,0.0,0.000
almas,0.000,0.0,0.197,0.000,0.000,0.0,0.000
...,...,...,...,...,...,...,...
varonil,0.000,0.0,0.000,0.000,0.205,0.0,0.000
victoria,0.000,0.0,0.000,0.000,0.205,0.0,0.000
victoriosa,0.000,0.0,0.000,0.000,0.000,0.0,0.214
virtud,0.000,0.0,0.000,0.217,0.000,0.0,0.000


In [60]:
from sklearn.metrics.pairwise import cosine_distances

dist_cos = cosine_distances(tfidf_matrix.T.values)
dist_cos = pd.DataFrame(dist_cos, columns = tfidf_matrix.columns, index = tfidf_matrix.columns)
dist_cos

Estrofa,estrofa 1,estrofa 2,estrofa 3,estrofa 4,estrofa 5,estrofa 6,estrofa 7
Estrofa,,,,,,,
estrofa 1,0.000000,0.873882,0.893158,0.836649,0.842008,0.892742,0.811087
estrofa 2,0.873882,0.000000,0.818620,0.896292,0.862665,0.875503,0.856149
estrofa 3,0.893158,0.818620,0.000000,0.928452,0.893432,0.930026,0.896350
estrofa 4,0.836649,0.896292,0.928452,0.000000,0.870279,0.957944,0.895724
estrofa 5,0.842008,0.862665,0.893432,0.870279,0.000000,0.898317,0.858849
estrofa 6,0.892742,0.875503,0.930026,0.957944,0.898317,0.000000,0.930554
estrofa 7,0.811087,0.856149,0.896350,0.895724,0.858849,0.930554,0.000000


In [66]:
dist=[]
Estrofa1=[]
Estrofa2=[]
fila=[]
columna=[]
for i in range(7):
    for j in range(7):
        if i<j:
            fila.append(i)
            columna.append(j)
            dist.append(dist_cos.iloc[i,j])
            Estrofa1.append(dist_cos.columns[i])
            Estrofa2.append(dist_cos.columns[j])
            
df=pd.DataFrame({'fila':fila, 'columna':columna, 'Estrofa1':prin1, 'Estrofa2':prin2, 'distancia': dist})
print(df.sort_values(by=['distancia'], ascending=[False]))

    fila  columna   Estrofa1   Estrofa2  distancia
16     3        5  estrofa 4  estrofa 6   0.957944
20     5        6  estrofa 6  estrofa 7   0.930554
13     2        5  estrofa 3  estrofa 6   0.930026
11     2        3  estrofa 3  estrofa 4   0.928452
18     4        5  estrofa 5  estrofa 6   0.898317
14     2        6  estrofa 3  estrofa 7   0.896350
7      1        3  estrofa 2  estrofa 4   0.896292
17     3        6  estrofa 4  estrofa 7   0.895724
12     2        4  estrofa 3  estrofa 5   0.893432
1      0        2  estrofa 1  estrofa 3   0.893158
4      0        5  estrofa 1  estrofa 6   0.892742
9      1        5  estrofa 2  estrofa 6   0.875503
0      0        1  estrofa 1  estrofa 2   0.873882
15     3        4  estrofa 4  estrofa 5   0.870279
8      1        4  estrofa 2  estrofa 5   0.862665
19     4        6  estrofa 5  estrofa 7   0.858849
10     1        6  estrofa 2  estrofa 7   0.856149
3      0        4  estrofa 1  estrofa 5   0.842008
2      0        3  estrofa 1  e

### las estrofas mas parecidas son la 4 y la 6
